<a href="https://colab.research.google.com/github/a150730/NLP-projects/blob/main/P2_Twitter_Sentiment_Analysis_Malay_Language_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Import Module and package
---

In [88]:
import pandas as pd
import re
import pickle
import numpy as np
from csv import reader
import nltk
from nltk.corpus import stopwords

# 2) Load Collected Raw Data
---

> The data topic and information details can be refered on Part 1 -3.1 of this project.

In [2]:
# dfKG = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/1_KerajaaanGagal_April29.csv')
#dfMSSP = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/2_MakeSchoolASaferPlace_April29.csv')
# dfKN = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/3_KRINanggala402_April29.csv')
# dfSH = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/4_sinarharian_April29.csv')
# dfCM = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/5_COVIDMalaysia_April29.csv')
#dfKJK = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/6_KitaJagaKita_April29.csv')
# dfJG = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/7_JusticeforGanapathy_April29.csv')
# dfUPSR = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/8_UPSR_April29.csv')
# dfARN = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/9_AstroRadioNews_April29.csv')

### Test sentence string
> * This few sample test is use just to test our model before proceed with raw dataset, to detect failure in line of code and etc...
* The sample test text was selected from extracted raw dataset. the sample est should include all the main concern for our preprocessing purpose. It should contains noisy text (Uppercase, mispell word, @mention tag, #hashtag, emoji/emoticon, stopwords, Non-Malay word and etc...)

In [2]:
TS1 = '@Marina_Ibrahims Kebabian PN makin terserlah dan muhyiddin kimak \n#KerajaanGagal' #contain ,#tag, @mention, Uppercase extra character
TS2 = '@kuasasiswa you made my day!👏😂\nso .lma dh X rasa @ujbu57ubj_78BJ gmbira mcm ni 👍 \n#Keraja...' #contain misspell word, emoticon/emoji, English word
TS3 = "@Izzud1n Omg 😭🤚🏼 1234 like, if we doesn't even get good results for upsr. not that we couldn't do well in sekolah menengah. aigoo https://t.co/K4GiA9l1Iw" #contain link, emoticon/emoji, English word, 

# 3) Data PreProcessing
---

## 3.1) Overall idea

> For this project purpose I will be covered 2 procesing tools which is using re- Regular Expression (standard python module for handling both pattern and string) *and combination of other techniques*.This module provides regular expression matching operations similar to those found in Perl. more info check - https://docs.python.org/3/library/re.html.

> Next is just PreProcesisng using **Malaya** nltk .which is a toolkit that known for processing Malay Language. **Malaya** is a Natural-Language-Toolkit library for bahasa Malaysia, powered by Deep Learning Tensorflow. More info check - https://malaya.readthedocs.io/en/latest/index.html#

## 3.2) PreProcessing 1- reMixin

###The Strategy/Methodology
---
> reMixin preprocesing is my propose preprocessing step which the overall step is combination of RE module and other other technique; as follows:
* Lowercasing the character --using python string manipulation
* Removing :
  * #Hashtag;i.e) #KerajaanGagal
  * @Mention or/@user; i.e) @mohd25
  * link; i.e) https://t.co/iCT2wJuiNLy6uiwENsx3
* 2 option for emoji and emoticon either:
  * 1) Remove 
  * 2) Translate them into text (Preferable to maintain the information)
* Identify English word and Translate them to Malay word -- using Malaya transformer translation
* Spelling Correction using Malaya probability spelling correction tools
* Removing stopwards and removing shortform word ( because most of them is stopwords and does not really affect the for clasification) i.e; yg == yang; utk == untuk; Except following shortform:
  * 'x', 'tk'  == 'tak' or tidak' whcih means 'no'
  * 'takda' or 'xda' == ' tak ada' or 'tidak ada' means ' do not have'.
* Removing the NonAlphanumeric and special character -re
  


#### *stripNonAlphaNumeric(string)*

In [3]:
def stripNonAlphaNum(text):
  text = re.compile(r'\W+', re.UNICODE).split(text)   
  return ' '.join(text)

#### *StripHashTag(String)*

In [4]:
def stripHashTag(text):
  text = re.compile(r'#\S+', re.UNICODE).split(text)   
  return ' '.join(text)

#### *StripMention(String)*

In [5]:
def stripMention(text):
  text = re.compile(r'@\S+', re.UNICODE).split(text)   
  return ' '.join(text)

#### *StripHyperLink(String)*

In [6]:
def stripHyperLink(text):
  text = re.compile(r'https?:\/\/\S+', re.UNICODE).split(text)   
  return ' '.join(text)

#### *stripUnderscore(String)*

In [7]:
def stripUnderscore(text):
  text = re.sub(r'_', ' ', text)
  return text

#### *remove_emoji(string)*

In [8]:
def removeEmoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

#### *emojiToText(string)*

In [92]:
#COnvert/translate emoji to text
with open('/content/drive/MyDrive/Colab Notebooks/Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
    Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def emojiToText(text):
    for emoj in Emoji_Dict:
      text = re.sub(r'('+emoj+')', Emoji_Dict[emoj], text)          
    return text

#### *removeStopWord(string)*

In [99]:
stopWord = open('/content/drive/MyDrive/Colab Notebooks/MalayStopWord.csv', 'r').read().split()
def removeStopWord(text):
  text = text.split()
  text = [word for word in text if word not in stopWord]
  return ' '.join(text)


#### ***reMixin(string)***

In [100]:
def reMixin(text):
  text = text.lower()
  text = emojiToText(text) 
  text = stripHashTag(text)
  text = stripMention(text)
  text = stripHyperLink(text)
  text = stripNonAlphaNum(text)
  text = stripUnderscore(text)
  text = removeStopWord(text)
  return text

In [101]:
print(TS1)
print(reMixin(TS1))

@Marina_Ibrahims Kebabian PN makin terserlah dan muhyiddin kimak 
#KerajaanGagal
kebabian pn terserlah muhyiddin kimak


In [102]:
print(TS2)
print(reMixin(TS2))

@kuasasiswa you made my day!👏😂
so .lma dh X rasa @ujbu57ubj_78BJ gmbira mcm ni 👍 
#Keraja...
you made my day clapping hands face with tears of joy so lma dh x rasa gmbira mcm ni thumbs up
